In [3]:
###tensorflow keras
# 自定义模型
# 自定义层
# 添加early_stopping
# 控制每一个batch的输入












# # TensorFlow 2.0 中，我最喜欢的一点就是它提供了多个抽象化（abstraction）级别，让你可以根据自己的项目，挑选出最适合的级别。本文中，我将解读如何权衡创建神经网络的两种样式：

# # 第一种是符号式（symbolic），符号式 API也称作声明式（Declarative） API。，即你通过操作层次图来创建模型；
# # 第二种是命令式（imperative），即你通过扩展类来创建模型。

# # 一.符号式（symbolic）：也称作声明式（Declarative） API
#    （1）Sequential API，是针对堆栈图的 API
#         model = tf.keras.Sequential()
        
#         model.add(layers.Dense(32, activation='relu'))
#         model.add(layers.Dense(32, activation='relu'))
#         model.add(layers.Dense(10, activation='softmax'))
        
#         model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
#                      loss=tf.keras.losses.categorical_crossentropy,
#                      metrics=['accuracy'])
#         model.fit(train_x, train_y, batch_size=32, epochs=5)

#    （2）Functional API，是针对 DAG 的 API；
#         用 Functional API 创建多输入/多输出模型，可以操作非线性拓扑、共享层的模型以及有多个输入或输出的模型
#         input_x = tf.keras.Input(shape=(72,))
#         hidden1 = layers.Dense(32, activation='relu')(input_x)
#         hidden2 = layers.Dense(16, activation='relu')(hidden1)
#         pred = layers.Dense(10, activation='softmax')(hidden2)
        
#         model = tf.keras.Model(inputs=input_x, outputs=pred)
        
#         model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
#                      loss=tf.keras.losses.categorical_crossentropy,
#                      metrics=['accuracy'])
#         model.fit(train_x, train_y, batch_size=32, epochs=5)
#    （3）其它符号式API
#         TensorFlow v1 (及 Theano) 提供了一个层级更低得多的 API
    
    
    
    
#     符号式 API 的优势和局限性
# 优势

#     使用符号式 API 创建的模型，就是一个类似图形的数据架构，这就意味着你的模型可以接受监测或者进行汇总。

#     你可以将模型当成图像来为其绘制图表（使用 keras.utils.plot_model）；或者简单地使用 model.summary() 来呈现层、权重以及形状的描述。

#     **同样地，在将层拼接在一起时，开发库的设计者可以运行扩展的层兼容性检查（在创建模型时和执行模型之前）。

#     **这类似于在编译器中进行类型检查，可以极大地减少开发者的错误。

#     大多数的故障排除都会在模型自定义阶段而不是执行期间进行。你可以保障所有编译的模型都能正常运行，这也加速了迭代，并让故障排除变得更简单。

#     符号式模型提供了一个一致的 API，这就使得这些模型的重复使用和共享变得简单。例如，在迁移学习中，你可以访问中间层的神经元，从而从现有的神经元中创建新的模型，就像这样：

#     符号式模型由可自然地进行复制和克隆的数据架构进行定义。
#       **例如，Sequential API 和 Functional API 可以提供 model.get_config()，model.to_json()，model.save()，clone_model(model)，同时仅凭借数据架构就能够重新创建同样地模型（而不需要访问用来定义和训练模型的原始代码）。


        
#      虽然精心设计的 API 应该跟神经网络的心智模型匹配，但是跟我们作为一个程序员所有的心智模型进行匹配也同样重要。对于我们大多数程序员来说，这种心智模型就是命令式的编程样式。在符号式 API 中，你操作「声明式的张量」（这些张量是没有值的）来创建图表。Keras 的 Sequential API 和 Functional API「感觉像」命令性的，它们是在开发者没有意识到他们在用符号定义模型的情况下被设计出来的。

# 局限性
# **符号式 API 的当前一代，可以很好地适用于有**向无环图的模型创建，这可以满足绝大多数实际应用的需要，
# 然而现在也有一些特例无法匹配这个简洁的抽象化，**例如，树形循环神经网络和递归神经网络等动态网络。

# 这也是为什么 TensorFlow 要同时还提供命令式的模型创建 API 样式（上文中提到的子类化 API）。无论是使用 Sequential API 还是 Functional API，
# 你都会用到所有熟悉的层、初始化器以及优化器。
# **同时，这两类 API 是完全互操作的，
# 因此你可以混合并且搭配两者使用**（例如将一种模型嵌套到另一种模型中）。
# 你可以采用一个符号式模型并在**子类化模型中将它用作层，反之亦然。




# #二.命令式（Imperative）API  :也称作模型子类化（Model Subclassing） API
#    在命令式 API 中，你要像编写 NumPy 一样编写模型。用这种 API 创建模型感觉像是在开发面向对象的 Python
#     从一个开发者的角度，它工作的方法就是扩展由框架定义的模型类别，将模型中的层实例化，然后命令式地编写下模型的正向传递（forward pass），而反向传递（backward pass）是自动生成的。
#     class MyModel(tf.keras.Model):
#         #在 init 方法中创建层并将它们设置为类实例的属性
#         def __init__(self, num_classes=10):
#             super(MyModel, self).__init__(name='my_model')
#             self.num_classes = num_classes
#             self.layer1 = layers.Dense(32, activation='relu')
#             self.layer2 = layers.Dense(num_classes, activation='softmax')
#         #在 call 方法中定义前向传播
#         def call(self, inputs):
#             h1 = self.layer1(inputs)
#             out = self.layer2(h1)
#             return out

#         def compute_output_shape(self, input_shape):
#             shape = tf.TensorShapej(input_shape).as_list()
#             shape[-1] = self.num_classes
#             return tf.TensorShape(shape)

#     model = MyModel(num_classes=10)
#     model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
#                  loss=tf.keras.losses.categorical_crossentropy,
#                  metrics=['accuracy'])

#     model.fit(train_x, train_y, batch_size=16, epochs=5)
    
    
# 命令式 API 的优势和局限性
# 优势

# **正向传递（forward pass）以命令式的方法编写，这就使得用自己的实现来替换掉通过开发库实现的部分（例如一层、一个神经元后者一个损失函数）变得很容易。这种方式的编程也非常自然，并且是深入了解深度学习的基本要点的不错的方法。

# **这也让你快速地尝试新想法变得很容易（深度学习开发工作流会变得与面向对象的 Python 一样），同时对于研究人员来说尤其有帮助。

# **也可以很轻易地使用 Python 指定模型正向传递中的任意控制流。
#  ———————————————— 
    
    
# 局限性

# **当使用命令式 API 使，模型是由某个类别方法来进行定义的。这样的话，模型就不再是一个清晰的数据架构，而是一个不透明的字节码。这种 API 样式所获得的灵活性是以可用性和可重用性换来的。

# **故障排除发生在执行期间，而不是在定义模型之时。
#   使用这一 API 样式时，由于几乎不会对输入或者层间兼容性进行检查，因此大量的故障排除压力就从框架上转移到了开发者身上。
   
# 命令式模型很难进行重复利用。例如你无法使用一个一致的 API 去访问中间层或神经元

    
# 相反地，提取神经元的方法就是采用一种新的调用（或者前进）方法来编写一个新的类别。最开始的时候可能会觉得这个操作有趣又简单，但是如果没有标准的话就会积累成技术债（tech debt）。 

# 命令式模型也很难进行检测、复制和克隆。

# **例如，model.save()， model.get_config()，以及 clone_model 对于子类化的模型是不起作用的，
# 而 model.summary() 也只能给你层的列表（并且不会提供任何关于它们怎样进行连接的信息，因为这些信息是访问不了的）



# 总结
# TensorFlow 2.0 直接支持符号式 API 和命令式 API 两种样式，因此大家可以选择最适合自己项目的抽象化（复杂性）层级。

# 如果你的目标是易用、低预算，同时你倾向于将模型考虑称层次图，那就使用 Keras 的 Sequential API 或者 Functional API (就像拼装乐高积木一样) 和内建的训练循环。这种方法适用于大多数问题。

# 如果你偏好于将模型考虑成面向对象的 Python/Numpy 开发者，同时有限考虑模型的灵活性和可破解性，Keras 的 Subclassing 这样的 API 会比较适合你。

    
    
# 训练循环（Training Lloop）
#     自定义的模型无论是使用 Sequential API、Functional API 还是使用子类化样式，都可以用两种方式进行训练：

# (1)一种是使用内建的训练路径和损失函数（第一个示例讲到的，我们使用的是 model.fit 和 model.compile）；

# (2)另一种是定制更复杂的训练循环（例如，当你想要自行编写梯度裁剪代码时）或损失函数，你可以按照以下方法轻易实现：

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=500)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [2]:
#最常见的模型类型是层的堆叠：tf.keras.Sequential 模型
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# ##可以model.add()添加层，也可以直接将列表加进去

# model = tf.keras.Sequential([
# # Adds a densely-connected layer with 64 units to the model:
# layers.Dense(64, activation='relu'),
# # Add another:
# layers.Dense(64, activation='relu'),
# # Add a softmax layer with 10 output units:
# layers.Dense(10, activation='softmax')])

In [3]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              ##上面的优化函数才是对的，tf.train.AdamOptimizer不能用
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
#tf.data输入数据并进行训练
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(32)
dataset = dataset.repeat()
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.repeat()



In [ ]:
model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset, validation_steps=3)

In [6]:
#3.4评估与预测
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)

30/30 [==============================] - 0s 4ms/step - loss: 11.4649 - accuracy: 0.1000


[11.464878495534261, 0.1]

In [7]:
# predict
result = model.predict(test_x, batch_size=32)
print(result)


[[0.09627154 0.09594291 0.10508692 ... 0.10022056 0.10034806 0.09685513]
 [0.10083509 0.10153824 0.10806082 ... 0.0947385  0.09095731 0.09853249]
 [0.10834585 0.09854479 0.10336304 ... 0.10329025 0.09235978 0.09772509]
 ...
 [0.09684049 0.10333864 0.09633884 ... 0.09816224 0.10727387 0.10667883]
 [0.10437717 0.10256477 0.09875691 ... 0.09714945 0.1054528  0.0862847 ]
 [0.10715229 0.09713148 0.10328652 ... 0.09553403 0.08844198 0.0949506 ]]


In [8]:
# ####################################
# 4.构建高级模型
# 4.1函数式api
# tf.keras.Sequential 模型是层的简单堆叠，无法表示任意模型。使用 Keras 函数式 API 可以构建复杂的模型拓扑，例如：


In [9]:
input_x = tf.keras.Input(shape=(72,))
hidden1 = layers.Dense(32, activation='relu')(input_x)
hidden2 = layers.Dense(16, activation='relu')(hidden1)
pred = layers.Dense(10, activation='softmax')(hidden2)

model = tf.keras.Model(inputs=input_x, outputs=pred)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 437us/sample - loss: 11.7316 - accuracy: 0.0930
Epoch 2/5
1000/1000 [==============================] - 0s 140us/sample - loss: 11.5862 - accuracy: 0.0960
Epoch 3/5
1000/1000 [==============================] - 0s 122us/sample - loss: 11.5750 - accuracy: 0.1140
Epoch 4/5
1000/1000 [==============================] - 0s 131us/sample - loss: 11.5670 - accuracy: 0.0920
Epoch 5/5
1000/1000 [==============================] - 0s 136us/sample - loss: 11.5627 - accuracy: 0.1010


In [10]:
# ########################################
# '''
# 4.2模型子类化：自定义模型通过模型子类化实现
# 通过对 tf.keras.Model 进行子类化并定义您自己的前向传播来构建完全可自定义的模型。
# 在 init 方法中创建层并将它们设置为类实例的属性。在 call 方法中定义前向传播

In [11]:
class MyModel(tf.keras.Model):
    #在 init 方法中创建层并将它们设置为类实例的属性
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(num_classes, activation='softmax')
    #在 call 方法中定义前向传播,将之前的层按照前向传播连起来
    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out
    
    
    ###计算输出层的维度
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShapej(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

model = MyModel(num_classes=10)
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 559us/sample - loss: 11.7106 - accuracy: 0.1000
Epoch 2/5
1000/1000 [==============================] - 0s 264us/sample - loss: 11.6248 - accuracy: 0.1140
Epoch 3/5
1000/1000 [==============================] - 0s 244us/sample - loss: 11.5983 - accuracy: 0.1140
Epoch 4/5
1000/1000 [==============================] - 0s 266us/sample - loss: 11.5767 - accuracy: 0.1290
Epoch 5/5
1000/1000 [==============================] - 0s 252us/sample - loss: 11.5640 - accuracy: 0.1340


In [1]:
##自定义层通过层的子类化实现

In [6]:
'''
4.3自定义层
通过对 tf.keras.layers.Layer 进行子类化并实现以下方法来创建自定义层：
使用 add_weight 方法添加权重。





'''
class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
    

    #build：创建层的权重。
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape,
                                   initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)
    #call：定义前向传播。
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    #compute_output_shape：指定在给定输入形状的情况下如何计算层的输出形状。
    #或者，可以通过实现 get_config 方法和 from_config 类方法序列化层。
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
model = tf.keras.Sequential(
[
    MyLayer(10),
    layers.Activation('softmax')
])


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

        



W0904 01:20:53.330900 140521470113536 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 11.5042 - accuracy: 0.1020
Epoch 2/5
1000/1000 [==============================] - 0s 309us/sample - loss: 11.4948 - accuracy: 0.1070
Epoch 3/5
1000/1000 [==============================] - 0s 226us/sample - loss: 11.4892 - accuracy: 0.1120
Epoch 4/5
1000/1000 [==============================] - 0s 254us/sample - loss: 11.4837 - accuracy: 0.1240
Epoch 5/5
1000/1000 [==============================] - 0s 194us/sample - loss: 11.4804 - accuracy: 0.1260
